In [6]:
import utils
import torch
import torchvision
import torchvision.datasets as dset
import numpy
import matplotlib.pyplot as plt

import torchvision.transforms as transforms

channel = 3
im_size = (32, 32)
num_classes = 2
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
transform=transforms.Compose([
                        transforms.Resize(im_size),
                        transforms.CenterCrop(im_size),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=mean, std=std)])

avg_distances = []; min_distances = []; max_distances = []; std_distances = []

for u, condense_rate in enumerate([2], 0):

    if condense_rate == 2:
        selected_indices =  [33, 54, 143, 207]
        temp_set = torchvision.datasets.ImageFolder(root='~/data/celebA-condensed/gender',transform=transform)
        origin_set = torch.utils.data.Subset(temp_set, selected_indices)
        origin_loader = torch.utils.data.DataLoader(origin_set, batch_size=1,
                                          shuffle=False, num_workers=0)
    elif condense_rate == 10:
        selected_indices = [60, 100, 69, 116, 43, 66, 8, 103, 98, 92, 142, 191, 208, 201, 137, 144, 135, 215, 127, 128]     
        temp_set = torchvision.datasets.ImageFolder(root='~/data/celebA-condensed/gender', transform=transform)
        origin_set = torch.utils.data.Subset(temp_set, selected_indices)
        origin_loader = torch.utils.data.DataLoader(origin_set, batch_size=1,
                                          shuffle=False, num_workers=0)
    elif condense_rate == 100:
        selected_indices = [101, 3, 114, 69, 33, 6, 80, 40, 15, 68, 10, 23, 4, 87, 5, 85, 73, 8, 111, 35, 95, 82, 54, 92, 110, 79, 109, 107, 84, 43, 7, 47, 37, 2, 53, 90, 112, 39, 29, 88, 12, 77, 22, 26, 48, 113, 11, 118, 17, 60, 13, 27, 55, 104, 46, 0, 117, 86, 21, 98, 99, 16, 103, 115, 83, 42, 32, 58, 78, 106, 116, 1, 74, 100, 20, 91, 75, 45, 67, 9, 44, 49, 105, 94, 108, 102, 93, 96, 34, 66, 61, 65, 62, 76, 25, 31, 30, 28, 52, 89, 135, 211, 201, 133, 128, 202, 219, 204, 142, 140, 150, 163, 187, 138, 214, 157, 165, 169, 205, 146, 156, 184, 131, 171, 175, 182, 120, 190, 155, 200, 152, 162, 144, 122, 147, 168, 173, 136, 194, 189, 197, 199, 153, 213, 174, 125, 177, 206, 188, 196, 172, 130, 126, 129, 164, 215, 210, 132, 148, 139, 209, 121, 185, 176, 217, 207, 208, 216, 191, 134, 119, 178, 181, 179, 143, 183, 160, 186, 127, 170, 203, 161, 212, 218, 151, 137, 192, 198, 158, 193, 180, 159, 124, 154, 145, 166, 149, 195, 141, 123]
        temp_set = torchvision.datasets.ImageFolder(root='~/data/celebA-condensed/gender', transform=transform)
        origin_set = torch.utils.data.Subset(temp_set, selected_indices)
        origin_loader = torch.utils.data.DataLoader(origin_set, batch_size=1,
                                          shuffle=False, num_workers=0)


    set = dset.ImageFolder(root='~/data/reconstructed-celebA/GM%dipc'%(condense_rate), transform=transform)
    loader =  torch.utils.data.DataLoader(set, batch_size=1, shuffle=False, num_workers=0)
    
    lpipses = []
    psnrs = []
    mses = []
    for i, data in enumerate(loader, 0):
        unpack_image, unpack_label = data
        for j, ref_data in enumerate(origin_loader, 0):
            ref_image, ref_label = ref_data
            
            lpipses.append( utils.lpips_loss(unpack_image, ref_image))
            psnrs.append(utils.psnr(unpack_image, ref_image))
            mses.append(utils.mse(unpack_image, ref_image))
    
    print('condense %d'%(condense_rate))
    print('1. unpack vs origin')
    print('mse', numpy.average(mses), numpy.std(mses))
    print('lpips', numpy.average(lpipses), numpy.std(lpipses))
    print('psnr', numpy.average(psnrs), numpy.std(psnrs))
    print('----------')

    
    set = dset.ImageFolder(root='~/data/celebA-condensed/GM%dipc'%(condense_rate), transform=transform)
    loader =  torch.utils.data.DataLoader(set, batch_size=1, shuffle=False, num_workers=0)
    
    lpipses = []
    psnrs = []
    mses = []
    for i, data in enumerate(loader, 0):
        unpack_image, unpack_label = data
        for j, ref_data in enumerate(origin_loader, 0):
            ref_image, ref_label = ref_data
            
            lpipses.append( utils.lpips_loss(unpack_image, ref_image))
            psnrs.append(utils.psnr(unpack_image, ref_image))
            mses.append(utils.mse(unpack_image, ref_image))
    
    print('2. condense vs origin')
    print('mse', numpy.average(mses), numpy.std(mses))
    print('lpips', numpy.average(lpipses), numpy.std(lpipses))
    print('psnr', numpy.average(psnrs), numpy.std(psnrs))
    print('----------')


condense 2
1. unpack vs origin
mse 0.2751078174139063 0.05601899109663031
lpips 0.1429336187429726 0.03427126262136974
psnr 5.685037732124329 0.8133949583389888
----------
2. condense vs origin
mse 0.24658280704170465 0.06683682684565213
lpips 0.13636809377931058 0.03676743100734499
psnr 6.257238000631332 1.2915540451685923
----------
